In [1]:
import numpy as np
import pandas as pd
import math
import nltk
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle

# Part A: Model Code and Exploration (100 pts)

## 1. Perform Exploratory Data Analysis (EDA) and discuss the data and what you observe prior to beginning modeling and how impact how to proceed [10 pts]

### (1) Read in both datasets, shuffle the training, reset index for the testing, and append together. Print a list of columns for exploration.

In [2]:
df8k = pd.read_csv('8k_diabetes.csv')
print('Number of variables:', len(list(df8k.columns)))
print(list(df8k.columns))

Number of variables: 51
['race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide.metformin', 'glipizide.metformin', 'glimepiride.pioglitazone', 'metformin.rosiglitazone', 'metformin.pioglitazone', 'change', 'diabetesMed', 'readmitted', 'diag_1_desc', 'diag_2_desc', 'diag_3_desc']


In [3]:
# shuffle 8k patients before appending with 2k patients
df8k_s = shuffle(df8k)
df8k_s['i'] = range(0,8000)
df8k_idx = df8k_s.set_index('i')

In [4]:
# append 2k patients with 8k patients
df2k = pd.read_csv('2k_diabetes_scoring.csv')
df2k['i'] = range(8000,10000)
df2k = df2k.set_index('i')
df = df8k_idx.append(df2k)

### (2) Explore: Target, missing values
#### - Target: not a rare target

In [5]:
# Target
print(df['readmitted'].value_counts())
df["y"] = 0
df.loc[df["readmitted"]==True, 'y'] = 1
y = 'y'
df['y'].describe()

False    4822
True     3178
Name: readmitted, dtype: int64


count    10000.000000
mean         0.317800
std          0.465645
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: y, dtype: float64

#### - numerical data can be described
##### (1) luckily, none of them have missing values
##### (2) Requires normalization


In [6]:
print(list(df.describe().columns))
num_data = list(df.describe().columns)
num_data.remove('y')
df[num_data].describe()

['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'y']


,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.434700,43.078600,1.399200,15.563800,0.281700,0.115000,0.387300,7.025300
std,3.021597,19.453315,1.706438,8.391613,1.119406,0.649475,0.854267,2.020957
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,32.000000,0.000000,10.000000,0.000000,0.000000,0.000000,5.000000
50%,4.000000,44.000000,1.000000,14.000000,0.000000,0.000000,0.000000,7.000000
75%,6.000000,57.000000,2.000000,19.000000,0.000000,0.000000,0.000000,9.000000
max,14.000000,120.000000,6.000000,81.000000,36.000000,42.000000,10.000000,9.000000


#### - Missing Values: impute or create dummies 

In [7]:
# Missing values
mdict = df.isnull().sum().to_dict()
for i in mdict.keys():
    if mdict[i]!=0: print(i,"has", mdict[i], 'missing values;')

admission_type_id has 721 missing values;
discharge_disposition_id has 469 missing values;
admission_source_id has 936 missing values;
readmitted has 2000 missing values;
diag_1_desc has 2 missing values;
diag_2_desc has 59 missing values;
diag_3_desc has 208 missing values;


#### - "ID“ values: use One-Hot encoding, create extra dummy for the missing

In [8]:
# admission_type_id has 576 missing values;
# discharge_disposition_id has 373 missing values;
# admission_source_id has 750 missing values;

#### - Text data: concatenate 3 columns into 1, and use TF-IDF

In [9]:
text_data = ['diag_1_desc', 'diag_2_desc', 'diag_3_desc']
text_code = ['diag_1', 'diag_2', 'diag_3']
df[text_data].head()

,diag_1_desc,diag_2_desc,diag_3_desc
i,,,
0,"Obesity, unspecified",Diabetes mellitus without mention of complicat...,Coronary atherosclerosis of unspecified type o...
1,"Osteoarthrosis, generalized, site unspecified","Nervous system complication, unspecified",Postmyocardial infarction syndrome
2,Diabetes with peripheral circulatory disorders...,Cellulitis and abscess of face,Atherosclerosis of aorta
3,Acute pancreatitis,Intussusception,Diabetes mellitus without mention of complicat...
4,Closed fracture of intracapsular section of ne...,Paroxysmal supraventricular tachycardia,Sideroblastic anemia


#### - Categorical data and requires encoding
1. features without variations should be thrown away.
2. Ordinal Encoding: age, weight, max_glu_serum, A1Cresult, and features that indicate "up" and "down"
3. One-Hot Encoding: other features

In [10]:
cat_data = list(df.columns)
# features without variations should be thrown away
no_variation_data = ['glimepiride.pioglitazone', 'metformin.rosiglitazone',
                     'metformin.pioglitazone', 'acetohexamide', 'tolbutamide',
                     'miglitol', 'troglitazone', 'tolazamide', 
                     'examide', 'citoglipton', 'glipizide.metformin']
# tabulate categorical data with 'readmitted' 
for i in (['readmitted'] + ['y'] + num_data + text_data + no_variation_data):
    cat_data.remove(i)
print(cat_data)
for i in cat_data:
    print('--- * Category:', i)
    print(pd.crosstab(df['readmitted'], df[i]))

['race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'insulin', 'glyburide.metformin', 'change', 'diabetesMed']
--- * Category: race
race          ?  AfricanAmerican  Asian  Caucasian  Hispanic  Other
readmitted                                                         
False       140             1080     24       3418        94     66
True         40              559     19       2473        52     35
--- * Category: gender
gender      Female  Male
readmitted              
False         2596  2226
True          1718  1460
--- * Category: age
age         [0-10)  [10-20)  [20-30)  [30-40)  [40-50)  [50-60)  [60-70)  \
readmitted                                                                 
Fal

## 2. Pre-processed categorical data for use in the model and justified pre-processing method. Note this may be different for each algorithm you try. [10 pts]

In [11]:
# Ordinal Encoding
# Step 1: determine the orders in a meaningful way
age_cat = ['[0-10)','[10-20)','[20-30)','[30-40)','[40-50)','[50-60)','[60-70)','[70-80)','[80-90)','[90-100)']
weight_cat = ['[0-25)','[25-50)','[50-75)','?','[75-100)','[100-125)','[125-150)','[150-175)']
max_glu_serum_cat = ['None','Norm','>200', '>300']
A1Cresult_cat = ['Norm','None','>7','>8']
metformin_cat = ['Down','No','Steady','Up']
repaglinide_cat = ['Down','No','Steady','Up']
nateglinide_cat = ['Down','No','Steady','Up']
chlorpropamide_cat = ['Down','No','Steady','Up']
glimepiride_cat = ['Down','No','Steady','Up']
glipizide_cat = ['Down','No','Steady','Up']
glyburide_cat = ['Down','No','Steady','Up']
pioglitazone_cat = ['Down','No','Steady','Up']
rosiglitazone_cat = ['Down','No','Steady','Up']
acarbose_cat = ['Down','No','Steady','Up']
insulin_cat = ['Down','No','Steady','Up']
glyburide_metformin_cat = ['Down','No','Steady','Up']
orders_cat = [age_cat, weight_cat, max_glu_serum_cat, A1Cresult_cat, metformin_cat,
              repaglinide_cat, nateglinide_cat, chlorpropamide_cat, glimepiride_cat,
              glipizide_cat, glyburide_cat, pioglitazone_cat, rosiglitazone_cat,
              acarbose_cat, insulin_cat, glyburide_metformin_cat]
ord_cat_data = ["age", "weight", "max_glu_serum", "A1Cresult", "metformin",
              "repaglinide", "nateglinide", "chlorpropamide", "glimepiride",
              "glipizide", "glyburide", "pioglitazone", "rosiglitazone",
              "acarbose", "insulin", "glyburide.metformin"]
# Step 2: instantiate the encoder
ord_encoder = OrdinalEncoder(categories=orders_cat)
# Step 3: fit data to encoder and transform the data
cat_ord_encoded = ord_encoder.fit_transform(df[ord_cat_data])

In [12]:
# One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse=False)
cat_onehot_encoded = onehot_encoder.fit_transform(df[cat_data])

## 3. Pre-processed numerical data appropriately including handling missing data and justified methods used. Note this may be different for each algorithm you try. [10 pts]

#### (1) 8 features in df.describe() are numerical, they don't have any missing values;

In [13]:
# Numerical data: use describe()
print(df.describe())
num_data = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 
             'num_medications', 'number_outpatient', 'number_emergency',
             'number_inpatient', 'number_diagnoses']
minmaxscaler = MinMaxScaler()
df_num_minmax_scaled = minmaxscaler.fit_transform(df[num_data])
zscaler = StandardScaler()
df_num_z_scaled = zscaler.fit_transform(df[num_data])

       time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
count      10000.000000        10000.000000    10000.000000     10000.000000   
mean           4.434700           43.078600        1.399200        15.563800   
std            3.021597           19.453315        1.706438         8.391613   
min            1.000000            1.000000        0.000000         1.000000   
25%            2.000000           32.000000        0.000000        10.000000   
50%            4.000000           44.000000        1.000000        14.000000   
75%            6.000000           57.000000        2.000000        19.000000   
max           14.000000          120.000000        6.000000        81.000000   

       number_outpatient  number_emergency  number_inpatient  \
count       10000.000000      10000.000000      10000.000000   
mean            0.281700          0.115000          0.387300   
std             1.119406          0.649475          0.854267   
min             0.00000

In [14]:
# Numerical data as arrays under 2 scales
df_num_minmax_scaled.shape, df_num_z_scaled.shape

((10000, 8), (10000, 8))

## 4. Implement a model to make predictions using text data using tf-idf [20 pts]

#### - Text Data: 
##### - (1) Combine (diag_1_desc, diag_2_desc, diag_3_desc) into 1 column
##### - (2) TF-IDF

In [15]:
# (1) combine (diag_1_desc, diag_2_desc, diag_3_desc) into 1 column
df["diag_desc"] = df["diag_1_desc"].astype(str) + " " + df["diag_2_desc"].astype(str)+ " " + df["diag_3_desc"].astype(str)
df[["diag_1_desc","diag_2_desc", "diag_3_desc", 'diag_desc']].head()

,diag_1_desc,diag_2_desc,diag_3_desc,diag_desc
i,,,,
0,"Obesity, unspecified",Diabetes mellitus without mention of complicat...,Coronary atherosclerosis of unspecified type o...,"Obesity, unspecified Diabetes mellitus without..."
1,"Osteoarthrosis, generalized, site unspecified","Nervous system complication, unspecified",Postmyocardial infarction syndrome,"Osteoarthrosis, generalized, site unspecified ..."
2,Diabetes with peripheral circulatory disorders...,Cellulitis and abscess of face,Atherosclerosis of aorta,Diabetes with peripheral circulatory disorders...
3,Acute pancreatitis,Intussusception,Diabetes mellitus without mention of complicat...,Acute pancreatitis Intussusception Diabetes me...
4,Closed fracture of intracapsular section of ne...,Paroxysmal supraventricular tachycardia,Sideroblastic anemia,Closed fracture of intracapsular section of ne...


In [16]:
# (2) TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df["diag_desc"])
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df_text = pd.DataFrame(denselist, columns=feature_names)

In [17]:
# separate 8k and 2k data
X_train, X_test, y_train, y_test, num_train, num_test, cat_ord_train, cat_ord_test, cat_oh_train, cat_oh_test = train_test_split(df_text, 
                                                                                                                                 df[y],
                                                                                                                                 df_num_z_scaled, 
                                                                                                                                 cat_ord_encoded, 
                                                                                                                                 cat_onehot_encoded,
                                                                                                                                 test_size = 0.2, 
                                                                                                                                 shuffle = False)

In [18]:
# (3) Use different models to train and predict tf-idf values

## 3.1 For training data, split into 2 parts for model stacking
X1, X2, y1, y2 = train_test_split(X_train, y_train, test_size = 0.5, shuffle = False)
## 3.2 Define a function to experiment different combination of models in upstream prediction
def upstream_models(X1,X2,y1,y2,model1,model2):
    '''
    Returns AUC and upstream predictions
    '''
    m1, m2 = model1, model2
    fit1, fit2 = m1.fit(X1,y1.values.ravel()), m2.fit(X2,y2.values.ravel())
    m1_pred_t, m1_pred_f = np.hsplit(m1.predict_proba(X2), 2)
    m2_pred_t, m2_pred_f = np.hsplit(m2.predict_proba(X1), 2)
    
    pred_up = np.concatenate((m2_pred_f, m1_pred_f))
    y_up = np.concatenate((y1,y2))
    
    return metrics.roc_auc_score(y_up, pred_up), pred_up

In [19]:
## 3.3 list the models and experiment different combinations
models = [LogisticRegressionCV(cv=5, random_state=42, max_iter=5000),
          MLPClassifier(alpha=1, max_iter=10000),
          MultinomialNB(),
          RandomForestClassifier(criterion='entropy', max_depth=5, random_state=42)]
for m1 in models:
    for m2 in models:
        if m1!=m2:
            a,b = upstream_models(X1,X2,y1,y2,m1,m2)
            print('Using the following two models:', m1,m2,', AUC is:', a)

Using the following two models: LogisticRegressionCV(cv=5, max_iter=5000, random_state=42) MLPClassifier(alpha=1, max_iter=10000) , AUC is: 0.583832844480628
Using the following two models: LogisticRegressionCV(cv=5, max_iter=5000, random_state=42) MultinomialNB() , AUC is: 0.5719639297440747
Using the following two models: LogisticRegressionCV(cv=5, max_iter=5000, random_state=42) RandomForestClassifier(criterion='entropy', max_depth=5, random_state=42) , AUC is: 0.5820419652009264
Using the following two models: MLPClassifier(alpha=1, max_iter=10000) LogisticRegressionCV(cv=5, max_iter=5000, random_state=42) , AUC is: 0.5833457754329785
Using the following two models: MLPClassifier(alpha=1, max_iter=10000) MultinomialNB() , AUC is: 0.575185345956061
Using the following two models: MLPClassifier(alpha=1, max_iter=10000) RandomForestClassifier(criterion='entropy', max_depth=5, random_state=42) , AUC is: 0.5823995015503466
Using the following two models: MultinomialNB() LogisticRegressi

In [20]:
## 3.4 According to these results, I choose MLP and Logit as my upstream models
m1 = MLPClassifier(alpha=1, max_iter=10000)
m2 = LogisticRegressionCV(cv=5, max_iter=10000, random_state=42)
AUC_upstream, pred_upstream = upstream_models(X1,X2,y1,y2,m1,m2)

In [21]:
print('Upstream AUC is:', AUC_upstream)

Upstream AUC is: 0.5830904296152598


## 5. Use model stacking to incorporate tf-idf predictions for all 3 text fields in downstream algorithm which uses non-text features [20 pts]

In [22]:
# downstream features and downstream label
print(pred_upstream.shape, num_train.shape, cat_ord_train.shape, cat_oh_train.shape)
X_down = np.concatenate((pred_upstream, num_train, cat_ord_train, cat_oh_train), axis=1)
print('Shape of concatenated feature matrix:', X_down.shape)

(8000, 1) (8000, 8) (8000, 16) (8000, 1542)
Shape of concatenated feature matrix: (8000, 1567)


In [23]:
# re-use sfold function from assignment #1
def sFold(folds, data, labels, model):
    '''
    Return AUC and predictions
    '''
    assert len(data)==len(labels), 'Error: different lengths between data and labels'

    # shuffle and split data into K folds
    kf = KFold(n_splits=folds, shuffle=True)
    
    # repeat the model k times
    k_expected_labels = []
    k_predicted_labels = []
    
    for train_index, test_index in kf.split(labels):
        
        k_fold_train_fm = data[train_index]
        k_fold_train_tv = labels[train_index]
        k_fold_test_fm = data[test_index]
        k_fold_test_tv = labels[test_index]

        # train the model
        km = model
        km.fit(k_fold_train_fm, k_fold_train_tv)
        # make a prediction
        k_fold_y_pred = km.predict_proba(k_fold_test_fm)
        k_fold_true_pred, k_fold_false_pred = np.hsplit(k_fold_y_pred, 2)

        k_expected_labels += k_fold_test_tv.tolist()
        k_predicted_labels += k_fold_false_pred.flatten().tolist()
    
    return metrics.roc_auc_score(k_expected_labels, k_predicted_labels), np.array(k_predicted_labels)

## 6. Perform experimentation for multiple modeling algorithms and justify why you selected the experiments you chose [20 pts]

In [24]:
# experiment using sfold function
models = [MLPClassifier(alpha=1, max_iter=5000),
          XGBClassifier(objective='binary:logistic', n_estimators = 10, seed = 123),
          LogisticRegressionCV(cv=5, random_state=42, max_iter=5000),
          LogisticRegression(penalty='l2', random_state=42, max_iter=5000),
          LogisticRegression(penalty='l1', solver='liblinear', random_state=42, max_iter=5000)]
for m in models:
    for x in [X_down]:
        a, b = sFold(5, x, y_train, m)
        print('- Using model:',m,', AUC is:', a)

- Using model: MLPClassifier(alpha=1, max_iter=5000) , AUC is: 0.6838126412950503
- Using model: XGBClassifier(n_estimators=10, seed=123) , AUC is: 0.6659835584178765
- Using model: LogisticRegressionCV(cv=5, max_iter=5000, random_state=42) , AUC is: 0.6961399125416103
- Using model: LogisticRegression(max_iter=5000, random_state=42) , AUC is: 0.67651300064551
- Using model: LogisticRegression(max_iter=5000, penalty='l1', random_state=42,
                   solver='liblinear') , AUC is: 0.6808078742307324


## 7. Final model selection and discussion of your model choice and the model weaknesses (generally, where model doesn’t perform well, etc.) [10 pts]

- I am choosing Neural Network and Logistic Regression in my upstream model where the features are TF-IDF.
- I am choosing Logistic Classifier as my downstream model, where features are upstream predictions, non-text related features.

- Generally, tree-based models are not working well in this case study.
- The disadvantage of this case study is the small size of data.

# Part B: Model Performance (100 pts)

In [25]:
# 2k patients are pre-processed 
for i in [X_test, y_test, num_test, cat_ord_test, cat_oh_test]:
    print(i.shape)

(2000, 1078)
(2000,)
(2000, 8)
(2000, 16)
(2000, 1542)


In [26]:
# upstream
m1_pred_t_2k, m1_pred_f_2k = np.hsplit(m1.predict_proba(X_test), 2)
m2_pred_t_2k, m2_pred_f_2k = np.hsplit(m2.predict_proba(X_test), 2)
for i in [m1_pred_f_2k, m2_pred_f_2k]:
    print(i.shape)

(2000, 1)
(2000, 1)


In [27]:
X_down_2k = np.concatenate((0.5*(m1_pred_f_2k + m2_pred_f_2k), num_test, cat_ord_test, cat_oh_test), axis=1)

In [28]:
m = LogisticRegressionCV(cv=5, max_iter=5000, random_state=42)
m.fit(X_down, y_train)

LogisticRegressionCV(cv=5, max_iter=5000, random_state=42)

In [29]:
pred_t_2k, pred_f_2k = np.hsplit(m.predict_proba(X_down_2k), 2)

In [35]:
df_2k_pred = pd.DataFrame(pred_f_2k)
df_2k_pred.describe()

,0
count,2000.000000
mean,0.403786
std,0.163054
min,0.047883
25%,0.281689
50%,0.389562
75%,0.500848
max,0.986419


In [36]:
df_2k_pred.to_csv('prediction2k.csv')